<a href="https://colab.research.google.com/github/JakubPac/kurs_ml/blob/main/pca_wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
np.set_printoptions(precision = 4, suppress = True, edgeitems = 5, linewidth = 200)

In [2]:
df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data = df.iloc[:, 1:]
target = df.iloc[:,0]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
target.value_counts()

,count
0,
2,71
1,59
3,48


Podział na zbior treningowy i testowy

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(X_train.shape)
print(X_test.shape)

(133, 13)
(45, 13)


Standaryzacja

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_train_std[:5]

array([[ 1.2103, -0.5407, -0.3556, -0.7445,  0.5735,  1.0254,  1.6664, -0.3713,  0.842 ,  1.5988,  0.6597,  0.7316,  1.6302],
       [ 1.0742, -0.6772,  0.8707,  0.0511,  1.0481,  1.1427,  1.5175,  0.2764,  0.2044,  0.6651,  0.7013, -0.0329,  1.2259],
       [ 0.9504, -0.737 ,  1.151 ,  0.8161,  0.0989,  1.2265,  1.3579, -0.6142,  1.3734,  0.2998,  0.9508,  0.1691,  1.7079],
       [ 0.9999, -0.3956,  1.0809, -0.8363,  1.0481,  1.2265,  0.8687,  0.1954,  0.1336,  0.5433,  0.7013,  0.4864,  1.9878],
       [ 1.656 , -0.3785,  1.221 ,  0.0511,  1.3871,  0.8914,  1.241 , -0.2904,  0.6472,  0.5027,  0.4518,  0.0825,  1.6923]])

PCA

In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
X_train_pca = pca.fit_transform(X_train_std)
X_test_pca = pca.transform(X_test_std)
X_train_pca.shape

(133, 3)

In [11]:
X_test_pca.shape

(45, 3)

In [12]:
results = pd.DataFrame(data = {'explained_variance_ratio' : pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.358761,0.358761,1
1,0.209401,0.568162,2
2,0.101233,0.669395,3


In [13]:
fig = go.Figure(data = [go.Bar(x = results['component'], y = results['explained_variance_ratio'], name = 'explained varaiance ratio'),
                        go.Scatter(x = results['component'], y = results['cumulative'], name = 'cumulative')],
                layout = go.Layout(title = 'PCA', width = 900, template = 'plotly_dark'))
fig.show()

In [14]:
X_train_pca_df = pd.DataFrame(data = np.c_[X_train_pca, y_train], columns = ['pca_1', 'pca_2', 'pca_3', 'target'])
X_train_pca_df.head()

,pca_1,pca_2,pca_3,target
0,2.783835,1.805536,-0.878889,1.0
1,1.994521,1.809912,0.578891,1.0
2,2.582071,1.579319,1.246507,1.0
3,2.297386,1.980148,0.072294,1.0
4,2.214401,2.492360,0.542095,1.0


In [15]:
px.scatter_3d(X_train_pca_df, 'pca_1', 'pca_2', 'pca_3', color = 'target', template = 'plotly_dark', width = 1000)

In [16]:
X_train_pca[:5]

array([[ 2.7838,  1.8055, -0.8789],
       [ 1.9945,  1.8099,  0.5789],
       [ 2.5821,  1.5793,  1.2465],
       [ 2.2974,  1.9801,  0.0723],
       [ 2.2144,  2.4924,  0.5421]])

In [17]:
X_test_pca[:5]

array([[ 2.2614, -1.6669,  0.0408],
       [-3.0851,  0.1917, -1.0605],
       [ 0.5108, -0.7601,  1.6612],
       [-3.6246,  0.5527, -1.3907],
       [-2.7988,  0.4938,  0.7009]])